In [6]:
import pandas as pd
import numpy as np
from googlesearch import search
from bs4 import BeautifulSoup
import requests
import time
import os

In [9]:
def get_info(query):
    
    imdb = None
    
    while True:
        try:
            for url in search(query, stop=5, pause=2, only_standard=True):
                if 'www.imdb.com' in url:
                    imdb = url
                    break        
            if imdb:
                source = requests.get(imdb)
                soup = BeautifulSoup(source.text, 'lxml')
                break
            else:
                return 
        except:    
            time.sleep(20)
            continue
        
    
    lang = soup.find('h4', text='Language:')
    if lang:
        lang = lang.findNext('a').text

    director = soup.find('h4', text='Director:')
    if director:
        director = director.findNext('a').text
    else:
        director = soup.find('h4', text='Directors:')
        if director:
            director = director.parent.text
            director = director.split(':')[-1].strip()

    cast = soup.find('h4', text='Stars:')
    if cast:
        cast = cast.parent.text
        cast = cast.split('|')[0].split(':')[-1].strip()

    date = soup.find(title="See more release dates")
    if date:
        date = date.text.strip()

    country = soup.find('h4', text='Country:')
    if country:
        country = country.parent.text
        country = ', '.join([a.strip() for a in country.split(':')[-1].split('|')])

    runtime = soup.find('time')
    if runtime:
        runtime = runtime.text.strip()
    
    return [lang, director, cast, date, country, runtime]

In [ ]:
df = pd.read_excel("C:\Dataframes\EIDR.xlsx")
for i in range(38000, 38145):
    print("Progress: " + str(i))
    try:
        info = get_info(df['Query'][i])
        df['Original Language'][i] = info[0]
        df['Director'][i] = info[1]
        df['Actors'][i] = info[2]
        df['Release Date'][i] = info[3]
        df['Country Of Origin'][i] = info[4]
        df['Running Time'][i] = info[5]
    except:
        continue

df.to_excel("C:\Dataframes\EIDR.xlsx", index=False)